In [1]:
import pandas as pd
import numpy as np
from scipy.signal import welch
from scipy.signal import find_peaks
from scipy.integrate import trapz


# HRV

In [34]:
# Load RR intervals from the file
rr_intervals = pd.read_csv('PCS26_CTRL_RRIntervals.csv')  # Replace with your file name
rr_intervals = rr_intervals[' rr'].values  # Assuming the column is named 'RR_intervals'

# Calculate SDNN
sdnn = np.std(rr_intervals)

# Calculate RMSSD
differences = np.diff(rr_intervals)
rmssd = np.sqrt(np.mean(differences ** 2))

# Calculate Heart Rate
heart_rate = 60000 / np.mean(rr_intervals)  # Assuming RR intervals are in milliseconds

# Calculate LF and HF
time = np.cumsum(rr_intervals) / 1000.0  # Convert to seconds
fs = 4.0  # Sampling frequency (Hz)
f, psd = welch(rr_intervals, fs=fs, nperseg=1024)
lf_band = (0.04, 0.15)  # Low-frequency band
hf_band = (0.15, 0.4)   # High-frequency band
lf_power = np.trapz(psd[(f >= lf_band[0]) & (f <= lf_band[1])], f[(f >= lf_band[0]) & (f <= lf_band[1])])
hf_power = np.trapz(psd[(f >= hf_band[0]) & (f <= hf_band[1])], f[(f >= hf_band[0]) & (f <= hf_band[1])])
total_power = lf_power + hf_power
lf_percent = (lf_power / total_power) * 100
hf_percent = (hf_power / total_power) * 100
lf_hf_ratio = lf_power / hf_power

print(f"SDNN: {sdnn:.2f} ms")
print(f"RMSSD: {rmssd:.2f} ms")
print(f"Heart Rate: {heart_rate:.2f} bpm")
print(f"LF%: {lf_percent:.2f}%")
print(f"HF%: {hf_percent:.2f}%")
print(f"LF/HF Ratio: {lf_hf_ratio:.2f}")

SDNN: 44.87 ms
RMSSD: 40.62 ms
Heart Rate: 67.02 bpm
LF%: 43.82%
HF%: 56.18%
LF/HF Ratio: 0.78


c:\ProgramData\anaconda3\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 1024 is greater than input length  = 819, using nperseg = 819
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


# CPET

## Anaerobic Threshold

In [28]:

# Import CPET data for anaerobic threshold analysis
df_cpet = pd.read_csv("PCS15_V1_CPET.csv", header=0)

# Remove any spaces in the names of the columns
df_cpet.columns = df_cpet.columns.str.replace(' ', '')

# Display the first few rows
print(df_cpet.head())


# Assuming df_cpet has columns: 'VO2', 'VCO2', 'VE', 'VE/VO2', 'VE/VCO2'
# calculate the first derivative of VE/VO2 and VE/VCO2 to find the inflection points.

# Calculate the derivative of VE/VO2 and VE/VCO2
df_cpet['dVE_VO2'] = np.gradient(df_cpet['VE/VO2'])
df_cpet['dVE_VCO2'] = np.gradient(df_cpet['VE/VCO2'])

# Find peaks in the derivative which correspond to the anaerobic threshold
peaks_VO2 = find_peaks(df_cpet['dVE_VO2'])[0]
peaks_VCO2 = find_peaks(df_cpet['dVE_VCO2'])[0]

# Assuming the anaerobic threshold is at the first peak (can be adjusted based on the data)
at_VO2 = df_cpet['VO2'].iloc[peaks_VO2[0]] if len(peaks_VO2) > 0 else None

print(f"Anaerobic Threshold (VO2): {at_VO2}")


   TIME  VO2/kg   VO2  VCO2     VE   RER  O2Pulse  RR    Vt  HR  WorkR  HR.1  \
0  0:30     1.6  0.14  0.11  12.70  0.78        0  14  0.92   0      0     0   
1  1:05     4.5  0.38  0.29  11.11  0.76        0  12  0.94   0      0     0   
2  1:30     7.8  0.66  0.46  14.59  0.70        0  12  1.18   0      0     0   
3  2:03     8.4  0.71  0.50  16.04  0.70        0  16  1.00   0      0     0   
4  2:33     7.7  0.64  0.47  14.66  0.73        0  12  1.20   0      0     0   

   VE/VCO2  VE/VO2  PetCO2  
0      118      92      10  
1       39      29      25  
2       32      22      29  
3       32      23      29  
4       31      23      30  
Anaerobic Threshold (VO2): 0.71


# V2

In [18]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression

# Import and clean data
df_cpet = pd.read_csv("PCS26_CTRL_CPET.csv", header=0)
df_cpet.columns = df_cpet.columns.str.replace(' ', '')
print(df_cpet.head())

# Dictionary to store results
results = {}

def safe_max(series):
    clean_series = pd.to_numeric(series, errors='coerce').replace([np.inf, -np.inf], np.nan)
    return clean_series.max()

# Find row with peak VO2 for parameter calculations
if 'VO2' in df_cpet.columns:
    vo2_peak_abs = safe_max(df_cpet['VO2'])  # assume VO2 is already in L/min
    peak_vo2_row = df_cpet[df_cpet['VO2'] == vo2_peak_abs].iloc[0] if not pd.isna(vo2_peak_abs) else None
else:
    peak_vo2_row = None

# Anaerobic Threshold Calculation (with %VO2peak)
try:
    if all(col in df_cpet.columns for col in ['VE/VO2', 'VE/VCO2', 'VO2']):
        df_cpet['dVE_VO2'] = np.gradient(df_cpet['VE/VO2'])
        df_cpet['dVE_VCO2'] = np.gradient(df_cpet['VE/VCO2'])
        
        peaks_VO2 = find_peaks(df_cpet['dVE_VO2'])[0]
        if len(peaks_VO2) > 0:
            at_vo2 = df_cpet['VO2'].iloc[peaks_VO2[0]]
            results['Anaerobic Threshold (VO2)'] = f"{at_vo2:.2f} L/min"
            if vo2_peak_abs and vo2_peak_abs > 0:
                at_pct = (at_vo2 / vo2_peak_abs) * 100
                results['Anaerobic Threshold (%VO2peak)'] = f"{at_pct:.1f} %"
        else:
            results['Anaerobic Threshold (VO2)'] = "Could not determine"
    else:
        missing = [col for col in ['VE/VO2', 'VE/VCO2', 'VO2'] if col not in df_cpet.columns]
        results['Anaerobic Threshold (VO2)'] = f"Missing columns: {', '.join(missing)}"
except Exception as e:
    results['Anaerobic Threshold (VO2)'] = f"Calculation error: {str(e)}"

# VO2 Peak Calculations
if 'VO2' in df_cpet.columns:
    results['VO2 Peak (absolute)'] = f"{vo2_peak_abs:.2f} L/min" if not pd.isna(vo2_peak_abs) else "Invalid VO2 data"
    if 'VO2/kg' in df_cpet.columns:
        vo2_peak_rel = peak_vo2_row['VO2/kg'] if peak_vo2_row is not None else safe_max(df_cpet['VO2/kg'])
        results['VO2 Peak (relative)'] = f"{vo2_peak_rel:.2f} mL/kg/min" if not pd.isna(vo2_peak_rel) else "Invalid VO2/kg data"
    else:
        results['VO2 Peak (relative)'] = "VO2/kg column missing"
else:
    results['VO2 Peak (absolute)'] = "VO2 column missing"
    results['VO2 Peak (relative)'] = "VO2 column missing"

# Parameter calculations at peak VO2
parameter_map = {
    'Peak HR': ('HR', 'bpm'),
    'Peak VE': ('VE', 'L/min'),
    'Peak VE/VO2': ('VE/VO2', ''),
    'Peak O2 Pulse': ('O2pulse', 'mL/beat'),
    'Peak RER': ('RER', ''),
    'Peak Respiratory Rate': ('RR', 'breaths/min'),
    'Peak PetCO2': ('PetCO2', 'mmHg'),
    'Peak Tidal Volume': ('Vt', 'L')
}

for param, (col, unit) in parameter_map.items():
    if col in df_cpet.columns:
        if peak_vo2_row is not None:
            value = peak_vo2_row[col]
            results[param] = f"{value:.2f} {unit}" if unit else f"{value:.2f}"
        else:
            value = safe_max(df_cpet[col]) if 'Peak' in param else df_cpet[col].iloc[-1]
            results[param] = f"{value:.2f} {unit}" if not pd.isna(value) else f"Invalid {col} data"
    else:
        results[param] = f"{col} column missing"

# Calculate O2_Pulse if not in original data but we have VO2 and HR
if 'O2_Pulse' not in df_cpet.columns and all(col in df_cpet.columns for col in ['VO2', 'HR']):
    df_cpet['O2_Pulse'] = np.where(df_cpet['HR'] > 0, df_cpet['VO2'] / df_cpet['HR'], np.nan)

# Calculate OUES (Oxygen Uptake Efficiency Slope)
if all(col in df_cpet.columns for col in ['VO2', 'VE']):
    df_oues = df_cpet[(df_cpet['VE'] > 0) & (df_cpet['VO2'] > 0)].copy()
    df_oues['log10_VE'] = np.log10(df_oues['VE'])

    try:
        X = df_oues['log10_VE'].values.reshape(-1, 1)
        y = df_oues['VO2'].values  # already in L/min
        reg = LinearRegression().fit(X, y)
        oues_slope = reg.coef_[0]
        r_squared = reg.score(X, y)

        results['OUES'] = f"{oues_slope:.3f} L/min (R²={r_squared:.3f})"
    except Exception as e:
        results['OUES'] = f"Calculation error: {str(e)}"
else:
    missing = [col for col in ['VO2', 'VE'] if col not in df_cpet.columns]
    results['OUES'] = f"Missing columns: {', '.join(missing)}"

# Print formatted results
print("\nCPET Analysis Results:")
max_len = max(len(k) for k in results.keys())
for param in sorted(results.keys()):
    print(f"{param.ljust(max_len)} : {results[param]}")


       TIME     VO2/kg       VO2      VCO2         VE       RER     RER.1  \
0  0.535333   7.665530  0.518956  0.341332  10.886865  0.657727  0.657727   
1  1.022333   9.333255  0.631861  0.382477  11.522161  0.605317  0.605317   
2  1.525500  11.291279  0.764420  0.466343  12.818426  0.610062  0.610062   
3  2.078667  11.852171  0.802392  0.523886  13.907189  0.652906  0.652906   
4  2.520167  10.912728  0.738792  0.506854  13.651427  0.686058  0.686058   

          RR        Vt     HR  WorkR   HR.1    VE/VCO2    VE/VO2       FEO2  \
0  16.811954  0.647567  102.0      0  102.0  31.895271  1.420236  15.598834   
1  14.373717  0.801613  103.0      0  103.0  30.125141  1.234528  14.868010   
2  11.924479  1.074967  105.0      0  105.0  27.487095  1.135250  14.329860   
3  16.269960  0.854777  110.0      0  110.0  26.546190  1.173388  14.482229   
4  11.325028  1.205421  107.0      0  107.0  26.933670  1.250964  14.837352   

      FECO2  
0  3.825042  
1  4.047659  
2  4.432268  
3  4.5

## NEW

In [19]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression

# Import and clean data
df_cpet = pd.read_csv("PCS01_V1_CPET.csv", header=0)
df_cpet.columns = df_cpet.columns.str.replace(' ', '')

# Dictionary to store results
results = {}

def safe_max(series):
    clean_series = pd.to_numeric(series, errors='coerce').replace([np.inf, -np.inf], np.nan)
    return clean_series.max()

# Convert VO2 to mL/min for O2 pulse calculation
if 'VO2' in df_cpet.columns:
    df_cpet['VO2_mL'] = df_cpet['VO2'] * 1000  # convert to mL/min

# Calculate O2 Pulse if missing and VO2 + HR available
if 'O2pulse' not in df_cpet.columns and all(col in df_cpet.columns for col in ['VO2', 'HR']):
    df_cpet['O2pulse'] = np.where(df_cpet['HR'] > 0, df_cpet['VO2_mL'] / df_cpet['HR'], np.nan)

# Calculate Respiratory Rate if missing and VE + Vt available
if 'RR' not in df_cpet.columns and all(col in df_cpet.columns for col in ['VE', 'Vt']):
    df_cpet['RR'] = np.where(df_cpet['Vt'] > 0, df_cpet['VE'] / df_cpet['Vt'], np.nan)

# Find row with peak VO2 for parameter calculations
if 'VO2' in df_cpet.columns:
    vo2_peak_abs = safe_max(df_cpet['VO2'])  # assume VO2 is already in L/min
    peak_vo2_row = df_cpet[df_cpet['VO2'] == vo2_peak_abs].iloc[0] if not pd.isna(vo2_peak_abs) else None
else:
    peak_vo2_row = None

# Anaerobic Threshold Calculation (with %VO2peak)
try:
    if all(col in df_cpet.columns for col in ['VE/VO2', 'VE/VCO2', 'VO2']):
        df_cpet['dVE_VO2'] = np.gradient(df_cpet['VE/VO2'])
        df_cpet['dVE_VCO2'] = np.gradient(df_cpet['VE/VCO2'])
        
        peaks_VO2 = find_peaks(df_cpet['dVE_VO2'])[0]
        if len(peaks_VO2) > 0:
            at_vo2 = df_cpet['VO2'].iloc[peaks_VO2[0]]
            results['Anaerobic Threshold (VO2)'] = f"{at_vo2:.2f} L/min"
            if vo2_peak_abs and vo2_peak_abs > 0:
                at_pct = (at_vo2 / vo2_peak_abs) * 100
                results['Anaerobic Threshold (%VO2peak)'] = f"{at_pct:.1f} %"
        else:
            results['Anaerobic Threshold (VO2)'] = "Could not determine"
    else:
        missing = [col for col in ['VE/VO2', 'VE/VCO2', 'VO2'] if col not in df_cpet.columns]
        results['Anaerobic Threshold (VO2)'] = f"Missing columns: {', '.join(missing)}"
except Exception as e:
    results['Anaerobic Threshold (VO2)'] = f"Calculation error: {str(e)}"

# VO2 Peak Calculations
if 'VO2' in df_cpet.columns:
    results['VO2 Peak (absolute)'] = f"{vo2_peak_abs:.2f} L/min" if not pd.isna(vo2_peak_abs) else "Invalid VO2 data"
    if 'VO2/kg' in df_cpet.columns:
        vo2_peak_rel = peak_vo2_row['VO2/kg'] if peak_vo2_row is not None else safe_max(df_cpet['VO2/kg'])
        results['VO2 Peak (relative)'] = f"{vo2_peak_rel:.2f} mL/kg/min" if not pd.isna(vo2_peak_rel) else "Invalid VO2/kg data"
    else:
        results['VO2 Peak (relative)'] = "VO2/kg column missing"
else:
    results['VO2 Peak (absolute)'] = "VO2 column missing"
    results['VO2 Peak (relative)'] = "VO2 column missing"

# Parameter calculations at peak VO2
parameter_map = {
    'Peak HR': ('HR', 'bpm'),
    'Peak VE': ('VE', 'L/min'),
    'Peak VE/VO2': ('VE/VO2', ''),
    'Peak O2 Pulse': ('O2pulse', 'mL/beat'),
    'Peak RER': ('RER', ''),
    'Peak Respiratory Rate': ('RR', 'breaths/min'),
    'Peak PetCO2': ('PetCO2', 'mmHg'),
    'Peak Tidal Volume': ('Vt', 'L')
}

for param, (col, unit) in parameter_map.items():
    if col in df_cpet.columns:
        if peak_vo2_row is not None:
            value = peak_vo2_row[col]
            results[param] = f"{value:.2f} {unit}" if unit else f"{value:.2f}"
        else:
            value = safe_max(df_cpet[col]) if 'Peak' in param else df_cpet[col].iloc[-1]
            results[param] = f"{value:.2f} {unit}" if not pd.isna(value) else f"Invalid {col} data"
    else:
        results[param] = f"{col} column missing"

# Calculate OUES (Oxygen Uptake Efficiency Slope)
if all(col in df_cpet.columns for col in ['VO2', 'VE']):
    df_oues = df_cpet[(df_cpet['VE'] > 0) & (df_cpet['VO2'] > 0)].copy()
    df_oues['log10_VE'] = np.log10(df_oues['VE'])

    try:
        X = df_oues['log10_VE'].values.reshape(-1, 1)
        y = df_oues['VO2'].values  # already in L/min
        reg = LinearRegression().fit(X, y)
        oues_slope = reg.coef_[0]
        r_squared = reg.score(X, y)

        results['OUES'] = f"{oues_slope:.3f} L/min (R²={r_squared:.3f})"
    except Exception as e:
        results['OUES'] = f"Calculation error: {str(e)}"
else:
    missing = [col for col in ['VO2', 'VE'] if col not in df_cpet.columns]
    results['OUES'] = f"Missing columns: {', '.join(missing)}"

# Print formatted results
print("\nCPET Analysis Results:")
max_len = max(len(k) for k in results.keys())
for param in sorted(results.keys()):
    print(f"{param.ljust(max_len)} : {results[param]}")



CPET Analysis Results:
Anaerobic Threshold (%VO2peak) : 30.3 %
Anaerobic Threshold (VO2)      : 0.58 L/min
OUES                           : 1.911 L/min (R²=0.975)
Peak HR                        : 0.00 bpm
Peak O2 Pulse                  : nan mL/beat
Peak PetCO2                    : PetCO2 column missing
Peak RER                       : 0.94
Peak Respiratory Rate          : 33.08 breaths/min
Peak Tidal Volume              : 1.50 L
Peak VE                        : 49.71 L/min
Peak VE/VO2                    : 26.17
VO2 Peak (absolute)            : 1.90 L/min
VO2 Peak (relative)            : 24.99 mL/kg/min


In [20]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# --- Load and clean data ---
df_cpet = pd.read_csv("PCS01_V1_CPET.csv", header=0)
df_cpet.columns = df_cpet.columns.str.replace(' ', '')

# Dictionary to store results
results = {}

def safe_max(series):
    clean_series = pd.to_numeric(series, errors='coerce').replace([np.inf, -np.inf], np.nan)
    return clean_series.max()

# Peak VO2 detection
if 'VO2' in df_cpet.columns:
    vo2_peak_abs = safe_max(df_cpet['VO2'])
    peak_vo2_row = df_cpet[df_cpet['VO2'] == vo2_peak_abs].iloc[0] if not pd.isna(vo2_peak_abs) else None
else:
    peak_vo2_row = None
    vo2_peak_abs = None

# --- Ventilatory Equivalent Method with smoothing and VE/VCO2 check ---
def calculate_vent_equiv_AT(df):
    # Check required columns
    required_cols = ['VE/VO2', 'VE/VCO2', 'VO2']
    if not all(col in df.columns for col in required_cols):
        missing = [c for c in required_cols if c not in df.columns]
        return None, f"Missing columns: {', '.join(missing)}"
    
    # Smooth VE/VO2 and VE/VCO2
    df['VE/VO2_smooth'] = df['VE/VO2'].rolling(window=3, center=True).mean()
    df['VE/VCO2_smooth'] = df['VE/VCO2'].rolling(window=3, center=True).mean()
    
    # Drop NaNs created by rolling
    df_clean = df.dropna(subset=['VE/VO2_smooth', 'VE/VCO2_smooth', 'VO2']).reset_index(drop=True)
    
    # Calculate gradients
    dVE_VO2 = np.gradient(df_clean['VE/VO2_smooth'])
    dVE_VCO2 = np.gradient(df_clean['VE/VCO2_smooth'])
    
    peaks = find_peaks(dVE_VO2)[0]
    
    at_vo2 = None
    for peak in peaks:
        # Check VE/VCO2 slope before and after peak
        start = max(peak - 3, 0)
        end = min(peak + 4, len(dVE_VCO2))
        pre_slope = np.mean(dVE_VCO2[start:peak+1])
        post_slope = np.mean(dVE_VCO2[peak:end])
        
        # Accept if VE/VCO2 slope does NOT increase
        if post_slope <= pre_slope:
            at_vo2 = df_clean['VO2'].iloc[peak]
            break

    if at_vo2:
        return at_vo2, None
    else:
        return None, "Could not determine AT via ventilatory equivalent method"

# Calculate ventilatory equivalent AT
at_vent_eq, vent_eq_err = calculate_vent_equiv_AT(df_cpet)
if at_vent_eq:
    results['Anaerobic Threshold (VO2)'] = f"{at_vent_eq:.2f} L/min"
    if vo2_peak_abs and vo2_peak_abs > 0:
        results['Anaerobic Threshold (%VO2peak)'] = f"{(at_vent_eq / vo2_peak_abs) * 100:.1f} %"
else:
    results['Anaerobic Threshold (VO2)'] = vent_eq_err or "Error in calculation"
    results['Anaerobic Threshold (%VO2peak)'] = vent_eq_err or "Error in calculation"

# --- Modified V-Slope Method ---
def calculate_vslope_AT(df):
    if not all(c in df.columns for c in ['VO2', 'VCO2']):
        return None
    df_vs = df[['VO2', 'VCO2']].dropna().reset_index(drop=True)
    n = len(df_vs)
    if n < 10:
        return None
    best_mse = np.inf
    best_idx = None
    for i in range(int(n * 0.2), int(n * 0.8)):
        X1 = df_vs['VO2'][:i].values.reshape(-1, 1)
        y1 = df_vs['VCO2'][:i].values
        X2 = df_vs['VO2'][i:].values.reshape(-1, 1)
        y2 = df_vs['VCO2'][i:].values
        
        reg1 = LinearRegression().fit(X1, y1)
        reg2 = LinearRegression().fit(X2, y2)
        
        pred1 = reg1.predict(X1)
        pred2 = reg2.predict(X2)
        
        mse = mean_squared_error(y1, pred1) + mean_squared_error(y2, pred2)
        
        if mse < best_mse:
            best_mse = mse
            best_idx = i
    
    if best_idx:
        return df_vs['VO2'].iloc[best_idx]
    return None

at_vslope = calculate_vslope_AT(df_cpet)
if at_vslope:
    results['AT (V-slope, absolute)'] = f"{at_vslope:.2f} L/min"
    if vo2_peak_abs and vo2_peak_abs > 0:
        results['AT (V-slope, %VO2peak)'] = f"{(at_vslope / vo2_peak_abs) * 100:.1f} %"
else:
    results['AT (V-slope, absolute)'] = "Could not determine"
    results['AT (V-slope, %VO2peak)'] = "Could not determine"

# --- RER Method (RER >= 1.0 and stays >=1.0) ---
def calculate_rer_AT(df):
    if 'RER' not in df.columns or 'VO2' not in df.columns:
        return None
    rer = df['RER'].values
    vo2 = df['VO2'].values
    for i in range(len(rer)):
        if rer[i] >= 1.0:
            if np.all(rer[i:] >= 1.0):
                return vo2[i]
    return None

at_rer = calculate_rer_AT(df_cpet)
if at_rer:
    results['AT (RER, absolute)'] = f"{at_rer:.2f} L/min"
    if vo2_peak_abs and vo2_peak_abs > 0:
        results['AT (RER, %VO2peak)'] = f"{(at_rer / vo2_peak_abs) * 100:.1f} %"
else:
    results['AT (RER, absolute)'] = "Could not determine"
    results['AT (RER, %VO2peak)'] = "Could not determine"

# --- VO2 Peak Reporting ---
if 'VO2' in df_cpet.columns:
    results['VO2 Peak (absolute)'] = f"{vo2_peak_abs:.2f} L/min" if not pd.isna(vo2_peak_abs) else "Invalid VO2 data"
    if 'VO2/kg' in df_cpet.columns:
        vo2_peak_rel = peak_vo2_row['VO2/kg'] if peak_vo2_row is not None else safe_max(df_cpet['VO2/kg'])
        results['VO2 Peak (relative)'] = f"{vo2_peak_rel:.2f} mL/kg/min" if not pd.isna(vo2_peak_rel) else "Invalid VO2/kg data"
    else:
        results['VO2 Peak (relative)'] = "VO2/kg column missing"
else:
    results['VO2 Peak (absolute)'] = "VO2 column missing"
    results['VO2 Peak (relative)'] = "VO2 column missing"

# --- Parameter calculations at peak VO2 ---
parameter_map = {
    'Peak HR': ('HR', 'bpm'),
    'Peak VE': ('VE', 'L/min'),
    'Peak VE/VO2': ('VE/VO2', ''),
    'Peak O2 Pulse': ('O2pulse', 'mL/beat'),
    'Peak RER': ('RER', ''),
    'Peak Respiratory Rate': ('RR', 'breaths/min'),
    'Peak PetCO2': ('PetCO2', 'mmHg'),
    'Peak Tidal Volume': ('Vt', 'L')
}

for param, (col, unit) in parameter_map.items():
    if col in df_cpet.columns:
        if peak_vo2_row is not None:
            value = peak_vo2_row[col]
            results[param] = f"{value:.2f} {unit}" if unit else f"{value:.2f}"
        else:
            value = safe_max(df_cpet[col]) if 'Peak' in param else df_cpet[col].iloc[-1]
            results[param] = f"{value:.2f} {unit}" if not pd.isna(value) else f"Invalid {col} data"
    else:
        results[param] = f"{col} column missing"

# --- Calculate O2 Pulse if missing but VO2 and HR exist ---
if 'O2pulse' not in df_cpet.columns and all(col in df_cpet.columns for col in ['VO2', 'HR']):
    df_cpet['O2pulse'] = np.where(df_cpet['HR'] > 0, df_cpet['VO2'] / df_cpet['HR'], np.nan)

# Calculate Respiratory Rate if missing and VE + Vt available
if 'RR' not in df_cpet.columns and all(col in df_cpet.columns for col in ['VE', 'Vt']):
    df_cpet['RR'] = np.where(df_cpet['Vt'] > 0, df_cpet['VE'] / df_cpet['Vt'], np.nan)


# --- Calculate OUES (Oxygen Uptake Efficiency Slope) ---
if all(col in df_cpet.columns for col in ['VO2', 'VE']):
    df_oues = df_cpet[(df_cpet['VE'] > 0) & (df_cpet['VO2'] > 0)].copy()
    df_oues['log10_VE'] = np.log10(df_oues['VE'])

    try:
        X = df_oues['log10_VE'].values.reshape(-1, 1)
        y = df_oues['VO2'].values  # already in L/min
        reg = LinearRegression().fit(X, y)
        oues_slope = reg.coef_[0]
        r_squared = reg.score(X, y)

        results['OUES'] = f"{oues_slope:.3f} L/min (R²={r_squared:.3f})"
    except Exception as e:
        results['OUES'] = f"Calculation error: {str(e)}"
else:
    missing = [col for col in ['VO2', 'VE'] if col not in df_cpet.columns]
    results['OUES'] = f"Missing columns: {', '.join(missing)}"

# --- Print all results ---
print("\nCPET Analysis Results:")
max_len = max(len(k) for k in results.keys())
for param in sorted(results.keys()):
    print(f"{param.ljust(max_len)} : {results[param]}")



CPET Analysis Results:
AT (RER, %VO2peak)             : Could not determine
AT (RER, absolute)             : Could not determine
AT (V-slope, %VO2peak)         : 57.5 %
AT (V-slope, absolute)         : 1.09 L/min
Anaerobic Threshold (%VO2peak) : 32.1 %
Anaerobic Threshold (VO2)      : 0.61 L/min
OUES                           : 1.911 L/min (R²=0.975)
Peak HR                        : 0.00 bpm
Peak O2 Pulse                  : O2pulse column missing
Peak PetCO2                    : PetCO2 column missing
Peak RER                       : 0.94
Peak Respiratory Rate          : RR column missing
Peak Tidal Volume              : 1.50 L
Peak VE                        : 49.71 L/min
Peak VE/VO2                    : 26.17
VO2 Peak (absolute)            : 1.90 L/min
VO2 Peak (relative)            : 24.99 mL/kg/min


## 3

In [21]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# --- Load and clean data ---
df_cpet = pd.read_csv("PCS01_V1_CPET.csv", header=0)
df_cpet.columns = df_cpet.columns.str.replace(' ', '')

# Dictionary to store results
results = {}

def safe_max(series):
    clean_series = pd.to_numeric(series, errors='coerce').replace([np.inf, -np.inf], np.nan)
    return clean_series.max()

# Convert VO2 to mL/min for O2 pulse calculation
if 'VO2' in df_cpet.columns:
    df_cpet['VO2_mL'] = df_cpet['VO2'] * 1000  # convert to mL/min

# Calculate O2 Pulse if missing and VO2 + HR available
if 'O2pulse' not in df_cpet.columns and all(col in df_cpet.columns for col in ['VO2_mL', 'HR']):
    df_cpet['O2pulse'] = np.where(df_cpet['HR'] > 0, df_cpet['VO2_mL'] / df_cpet['HR'], np.nan)

# Calculate Respiratory Rate if missing and VE + Vt available
if 'RR' not in df_cpet.columns and all(col in df_cpet.columns for col in ['VE', 'Vt']):
    df_cpet['RR'] = np.where(df_cpet['Vt'] > 0, df_cpet['VE'] / df_cpet['Vt'], np.nan)

# Peak VO2 detection
if 'VO2' in df_cpet.columns:
    vo2_peak_abs = safe_max(df_cpet['VO2'])
    peak_vo2_row = df_cpet[df_cpet['VO2'] == vo2_peak_abs].iloc[0] if not pd.isna(vo2_peak_abs) else None
else:
    peak_vo2_row = None
    vo2_peak_abs = None

# --- Ventilatory Equivalent Method with smoothing and VE/VCO2 check ---
def calculate_vent_equiv_AT(df):
    required_cols = ['VE/VO2', 'VE/VCO2', 'VO2']
    if not all(col in df.columns for col in required_cols):
        missing = [c for c in required_cols if c not in df.columns]
        return None, f"Missing columns: {', '.join(missing)}"
    
    df['VE/VO2_smooth'] = df['VE/VO2'].rolling(window=3, center=True).mean()
    df['VE/VCO2_smooth'] = df['VE/VCO2'].rolling(window=3, center=True).mean()
    
    df_clean = df.dropna(subset=['VE/VO2_smooth', 'VE/VCO2_smooth', 'VO2']).reset_index(drop=True)
    
    dVE_VO2 = np.gradient(df_clean['VE/VO2_smooth'])
    dVE_VCO2 = np.gradient(df_clean['VE/VCO2_smooth'])
    
    peaks = find_peaks(dVE_VO2)[0]
    
    at_vo2 = None
    for peak in peaks:
        start = max(peak - 3, 0)
        end = min(peak + 4, len(dVE_VCO2))
        pre_slope = np.mean(dVE_VCO2[start:peak+1])
        post_slope = np.mean(dVE_VCO2[peak:end])
        
        if post_slope <= pre_slope:
            at_vo2 = df_clean['VO2'].iloc[peak]
            break

    if at_vo2:
        return at_vo2, None
    else:
        return None, "Could not determine AT via ventilatory equivalent method"

at_vent_eq, vent_eq_err = calculate_vent_equiv_AT(df_cpet)
if at_vent_eq:
    results['Anaerobic Threshold (VO2)'] = f"{at_vent_eq:.2f} L/min"
    if vo2_peak_abs and vo2_peak_abs > 0:
        results['Anaerobic Threshold (%VO2peak)'] = f"{(at_vent_eq / vo2_peak_abs) * 100:.1f} %"
else:
    results['Anaerobic Threshold (VO2)'] = vent_eq_err or "Error in calculation"
    results['Anaerobic Threshold (%VO2peak)'] = vent_eq_err or "Error in calculation"

# --- Modified V-Slope Method ---
def calculate_vslope_AT(df):
    if not all(c in df.columns for c in ['VO2', 'VCO2']):
        return None
    df_vs = df[['VO2', 'VCO2']].dropna().reset_index(drop=True)
    n = len(df_vs)
    if n < 10:
        return None
    best_mse = np.inf
    best_idx = None
    for i in range(int(n * 0.2), int(n * 0.8)):
        X1 = df_vs['VO2'][:i].values.reshape(-1, 1)
        y1 = df_vs['VCO2'][:i].values
        X2 = df_vs['VO2'][i:].values.reshape(-1, 1)
        y2 = df_vs['VCO2'][i:].values
        
        reg1 = LinearRegression().fit(X1, y1)
        reg2 = LinearRegression().fit(X2, y2)
        
        pred1 = reg1.predict(X1)
        pred2 = reg2.predict(X2)
        
        mse = mean_squared_error(y1, pred1) + mean_squared_error(y2, pred2)
        
        if mse < best_mse:
            best_mse = mse
            best_idx = i
    
    if best_idx:
        return df_vs['VO2'].iloc[best_idx]
    return None

at_vslope = calculate_vslope_AT(df_cpet)
if at_vslope:
    results['AT (V-slope, absolute)'] = f"{at_vslope:.2f} L/min"
    if vo2_peak_abs and vo2_peak_abs > 0:
        results['AT (V-slope, %VO2peak)'] = f"{(at_vslope / vo2_peak_abs) * 100:.1f} %"
else:
    results['AT (V-slope, absolute)'] = "Could not determine"
    results['AT (V-slope, %VO2peak)'] = "Could not determine"

# --- RER Method (RER >= 1.0 and stays >=1.0) ---
def calculate_rer_AT(df):
    if 'RER' not in df.columns or 'VO2' not in df.columns:
        return None
    rer = df['RER'].values
    vo2 = df['VO2'].values
    for i in range(len(rer)):
        if rer[i] >= 1.0:
            if np.all(rer[i:] >= 1.0):
                return vo2[i]
    return None

at_rer = calculate_rer_AT(df_cpet)
if at_rer:
    results['AT (RER, absolute)'] = f"{at_rer:.2f} L/min"
    if vo2_peak_abs and vo2_peak_abs > 0:
        results['AT (RER, %VO2peak)'] = f"{(at_rer / vo2_peak_abs) * 100:.1f} %"
else:
    results['AT (RER, absolute)'] = "Could not determine"
    results['AT (RER, %VO2peak)'] = "Could not determine"

# --- VO2 Peak Reporting ---
if 'VO2' in df_cpet.columns:
    results['VO2 Peak (absolute)'] = f"{vo2_peak_abs:.2f} L/min" if not pd.isna(vo2_peak_abs) else "Invalid VO2 data"
    if 'VO2/kg' in df_cpet.columns:
        vo2_peak_rel = peak_vo2_row['VO2/kg'] if peak_vo2_row is not None else safe_max(df_cpet['VO2/kg'])
        results['VO2 Peak (relative)'] = f"{vo2_peak_rel:.2f} mL/kg/min" if not pd.isna(vo2_peak_rel) else "Invalid VO2/kg data"
    else:
        results['VO2 Peak (relative)'] = "VO2/kg column missing"
else:
    results['VO2 Peak (absolute)'] = "VO2 column missing"
    results['VO2 Peak (relative)'] = "VO2 column missing"

# --- Parameter calculations at peak VO2 ---
parameter_map = {
    'Peak HR': ('HR', 'bpm'),
    'Peak VE': ('VE', 'L/min'),
    'Peak VE/VO2': ('VE/VO2', ''),
    'Peak O2 Pulse': ('O2pulse', 'mL/beat'),
    'Peak RER': ('RER', ''),
    'Peak Respiratory Rate': ('RR', 'breaths/min'),
    'Peak PetCO2': ('PetCO2', 'mmHg'),
    'Peak Tidal Volume': ('Vt', 'L')
}

for param, (col, unit) in parameter_map.items():
    if col in df_cpet.columns:
        if peak_vo2_row is not None:
            value = peak_vo2_row[col]
            results[param] = f"{value:.2f} {unit}" if unit else f"{value:.2f}"
        else:
            value = safe_max(df_cpet[col]) if 'Peak' in param else df_cpet[col].iloc[-1]
            results[param] = f"{value:.2f} {unit}" if not pd.isna(value) else f"Invalid {col} data"
    else:
        results[param] = f"{col} column missing"

# --- Calculate OUES (Oxygen Uptake Efficiency Slope) ---
if all(col in df_cpet.columns for col in ['VO2', 'VE']):
    df_oues = df_cpet[(df_cpet['VE'] > 0) & (df_cpet['VO2'] > 0)].copy()
    df_oues['log10_VE'] = np.log10(df_oues['VE'])

    try:
        X = df_oues['log10_VE'].values.reshape(-1, 1)
        y = df_oues['VO2'].values
        reg = LinearRegression().fit(X, y)
        oues_slope = reg.coef_[0]
        r_squared = reg.score(X, y)

        results['OUES'] = f"{oues_slope:.3f} L/min (R²={r_squared:.3f})"
    except Exception as e:
        results['OUES'] = f"Calculation error: {str(e)}"
else:
    missing = [col for col in ['VO2', 'VE'] if col not in df_cpet.columns]
    results['OUES'] = f"Missing columns: {', '.join(missing)}"

# --- PetCO₂ Values at Specific Timepoints ---
# Initialize default values
petco2_at_peak = None
petco2_at_vslope = None
petco2_at_venteq = None

# Get PetCO₂ at VO2 Peak
if peak_vo2_row is not None and 'PetCO2' in df_cpet.columns:
    petco2_at_peak = peak_vo2_row['PetCO2']
    results['PetCO2 at VO2 Peak'] = f"{petco2_at_peak:.2f} mmHg"
else:
    results['PetCO2 at VO2 Peak'] = "Unavailable"

# Get PetCO₂ at AT (V-slope)
if at_vslope and 'PetCO2' in df_cpet.columns:
    vslope_idx = (df_cpet['VO2'] - at_vslope).abs().idxmin()
    petco2_at_vslope = df_cpet.loc[vslope_idx, 'PetCO2']
    results['PetCO2 at AT (V-slope)'] = f"{petco2_at_vslope:.2f} mmHg"
else:
    results['PetCO2 at AT (V-slope)'] = "Unavailable"

# Get PetCO₂ at AT (Ventilatory Equivalent Method)
if at_vent_eq and 'PetCO2' in df_cpet.columns:
    venteq_idx = (df_cpet['VO2'] - at_vent_eq).abs().idxmin()
    petco2_at_venteq = df_cpet.loc[venteq_idx, 'PetCO2']
    results['PetCO2 at AT (VEM)'] = f"{petco2_at_venteq:.2f} mmHg"
else:
    results['PetCO2 at AT (VEM)'] = "Unavailable"


# --- Print all results ---
print("\nCPET Analysis Results:")
max_len = max(len(k) for k in results.keys())
for param in sorted(results.keys()):
    print(f"{param.ljust(max_len)} : {results[param]}")



CPET Analysis Results:
AT (RER, %VO2peak)             : Could not determine
AT (RER, absolute)             : Could not determine
AT (V-slope, %VO2peak)         : 57.5 %
AT (V-slope, absolute)         : 1.09 L/min
Anaerobic Threshold (%VO2peak) : 32.1 %
Anaerobic Threshold (VO2)      : 0.61 L/min
OUES                           : 1.911 L/min (R²=0.975)
Peak HR                        : 0.00 bpm
Peak O2 Pulse                  : nan mL/beat
Peak PetCO2                    : PetCO2 column missing
Peak RER                       : 0.94
Peak Respiratory Rate          : 33.08 breaths/min
Peak Tidal Volume              : 1.50 L
Peak VE                        : 49.71 L/min
Peak VE/VO2                    : 26.17
PetCO2 at AT (V-slope)         : Unavailable
PetCO2 at AT (VEM)             : Unavailable
PetCO2 at VO2 Peak             : Unavailable
VO2 Peak (absolute)            : 1.90 L/min
VO2 Peak (relative)            : 24.99 mL/kg/min


# 4

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# --- Load and clean data ---
df_cpet = pd.read_csv("PCS01_V1_CPET.csv")
df_cpet.columns = df_cpet.columns.str.replace(' ', '')

# --- Derived variables ---
df_cpet['VO2_mL'] = df_cpet['VO2'] * 1000
df_cpet['O2pulse'] = np.where(df_cpet['HR'] > 0, df_cpet['VO2_mL'] / df_cpet['HR'], np.nan)
df_cpet['RR'] = np.where(df_cpet['Vt'] > 0, df_cpet['VE'] / df_cpet['Vt'], np.nan)

# --- Ventilatory Equivalent Method with smoothing and VE/VCO2 check ---
def calculate_vent_equiv_AT(df):
    required_cols = ['VE/VO2', 'VE/VCO2', 'VO2']
    if not all(col in df.columns for col in required_cols):
        missing = [c for c in required_cols if c not in df.columns]
        return None, f"Missing columns: {', '.join(missing)}"
    
    df['VE/VO2_smooth'] = df['VE/VO2'].rolling(window=3, center=True).mean()
    df['VE/VCO2_smooth'] = df['VE/VCO2'].rolling(window=3, center=True).mean()
    
    df_clean = df.dropna(subset=['VE/VO2_smooth', 'VE/VCO2_smooth', 'VO2']).reset_index(drop=True)
    
    dVE_VO2 = np.gradient(df_clean['VE/VO2_smooth'])
    dVE_VCO2 = np.gradient(df_clean['VE/VCO2_smooth'])
    
    peaks = find_peaks(dVE_VO2)[0]
    
    at_vo2 = None
    for peak in peaks:
        start = max(peak - 3, 0)
        end = min(peak + 4, len(dVE_VCO2))
        pre_slope = np.mean(dVE_VCO2[start:peak+1])
        post_slope = np.mean(dVE_VCO2[peak:end])
        
        if post_slope <= pre_slope:
            at_vo2 = df_clean['VO2'].iloc[peak]
            break

    if at_vo2:
        return at_vo2, None
    else:
        return None, "Could not determine AT via ventilatory equivalent method"

at_vent_eq, vent_eq_err = calculate_vent_equiv_AT(df_cpet)
if at_vent_eq:
    results['Anaerobic Threshold (VO2)'] = f"{at_vent_eq:.2f} L/min"
    if vo2_peak_abs and vo2_peak_abs > 0:
        results['Anaerobic Threshold (%VO2peak)'] = f"{(at_vent_eq / vo2_peak_abs) * 100:.1f} %"
else:
    results['Anaerobic Threshold (VO2)'] = vent_eq_err or "Error in calculation"
    results['Anaerobic Threshold (%VO2peak)'] = vent_eq_err or "Error in calculation"

# --- Modified V-Slope Method ---
def calculate_vslope_AT(df):
    if not all(c in df.columns for c in ['VO2', 'VCO2']):
        return None
    df_vs = df[['VO2', 'VCO2']].dropna().reset_index(drop=True)
    n = len(df_vs)
    if n < 10:
        return None
    best_mse = np.inf
    best_idx = None
    for i in range(int(n * 0.2), int(n * 0.8)):
        X1 = df_vs['VO2'][:i].values.reshape(-1, 1)
        y1 = df_vs['VCO2'][:i].values
        X2 = df_vs['VO2'][i:].values.reshape(-1, 1)
        y2 = df_vs['VCO2'][i:].values
        
        reg1 = LinearRegression().fit(X1, y1)
        reg2 = LinearRegression().fit(X2, y2)
        
        pred1 = reg1.predict(X1)
        pred2 = reg2.predict(X2)
        
        mse = mean_squared_error(y1, pred1) + mean_squared_error(y2, pred2)
        
        if mse < best_mse:
            best_mse = mse
            best_idx = i
    
    if best_idx:
        return df_vs['VO2'].iloc[best_idx]
    return None

at_vslope = calculate_vslope_AT(df_cpet)
if at_vslope:
    results['AT (V-slope, absolute)'] = f"{at_vslope:.2f} L/min"
    if vo2_peak_abs and vo2_peak_abs > 0:
        results['AT (V-slope, %VO2peak)'] = f"{(at_vslope / vo2_peak_abs) * 100:.1f} %"
else:
    results['AT (V-slope, absolute)'] = "Could not determine"
    results['AT (V-slope, %VO2peak)'] = "Could not determine"

# --- Peak VO2 ---
vo2_peak_abs = df_cpet['VO2'].max()
vo2_peak_idx = df_cpet['VO2'].idxmax()

# --- OUES ---
def calculate_oues(df):
    df_filtered = df[(df['VO2'] > 0) & (df['VE'] > 0)].copy()
    df_filtered['logVE'] = np.log10(df_filtered['VE'])
    slope, intercept, r_value, _, _ = stats.linregress(df_filtered['logVE'], df_filtered['VO2'])
    return slope, r_value**2

oues_val, oues_r2 = calculate_oues(df_cpet)

# --- Peak Values at VO2 Peak Index ---
parameter_map = {
    'Peak HR': ('HR', 'bpm'),
    'Peak VE': ('VE', 'L/min'),
    'Peak VE/VO2': ('VE/VO2', ''),
    'Peak O2 Pulse': ('O2pulse', 'mL/beat'),
    'Peak RER': ('RER', ''),
    'Peak Respiratory Rate': ('RR', 'breaths/min'),
    'Peak Tidal Volume': ('Vt', 'L')
}

# --- Results Printout ---
results = {}

# AT: VE/VO2 Method
if at_venteq:
    results['AT (VE/VO2, absolute)'] = f"{at_venteq} L/min"
    results['AT (VE/VO2, %VO2peak)'] = f"{(at_venteq / vo2_peak_abs) * 100:.1f} %"
else:
    results['AT (VE/VO2, absolute)'] = "Could not determine"
    results['AT (VE/VO2, %VO2peak)'] = "Could not determine"

# AT: V-slope Method
if at_vslope:
    results['AT (V-slope, absolute)'] = f"{at_vslope} L/min"
    results['AT (V-slope, %VO2peak)'] = f"{(at_vslope / vo2_peak_abs) * 100:.1f} %"
else:
    results['AT (V-slope, absolute)'] = "Could not determine"
    results['AT (V-slope, %VO2peak)'] = "Could not determine"

# OUES
results['OUES'] = f"{oues_val:.3f} L/min (R²={oues_r2:.3f})"

# Peak VO2
results['VO2 Peak (absolute)'] = f"{vo2_peak_abs:.2f} L/min"
results['VO2 Peak (relative)'] = f"{(vo2_peak_abs * 1000 / 76):.2f} mL/kg/min"  # Replace 76 with actual weight

# Peak Parameters
for label, (col, unit) in parameter_map.items():
    val = df_cpet.loc[vo2_peak_idx, col]
    results[label] = f"{val:.2f} {unit}"

# --- PetCO2 Handling and Classifications ---
if 'PetCO2' in df_cpet.columns:
    petco2_vslope = df_cpet.loc[df_cpet['VO2'] >= at_vslope, 'PetCO2'].iloc[0] if at_vslope else np.nan
    petco2_venteq = df_cpet.loc[df_cpet['VO2'] >= at_venteq, 'PetCO2'].iloc[0] if at_venteq else np.nan
    petco2_peak = df_cpet.loc[vo2_peak_idx, 'PetCO2']

    # Report PetCO2
    results['PetCO2 at VO2 Peak'] = f"{petco2_peak:.1f}"
    results['PetCO2 at AT (V-slope)'] = f"{petco2_vslope:.1f}" if not np.isnan(petco2_vslope) else "N/A"
    results['PetCO2 at AT (VE/VO2)'] = f"{petco2_venteq:.1f}" if not np.isnan(petco2_venteq) else "N/A"
    
else:
    # If PetCO2 column is missing
    results['PetCO2 at VO2 Peak'] = "Column missing"
    results['PetCO2 at AT (V-slope)'] = "Column missing"
    results['PetCO2 at AT (VE/VO2)'] = "Column missing"

# --- Final Output ---
print("🔍 CPET Analysis Results:")
for k, v in results.items():
    print(f"{k:<35}: {v}")

🔍 CPET Analysis Results:
AT (VE/VO2, absolute)              : 0.575460732 L/min
AT (VE/VO2, %VO2peak)              : 30.3 %
AT (V-slope, absolute)             : 1.092734814 L/min
AT (V-slope, %VO2peak)             : 57.5 %
OUES                               : 1.911 L/min (R²=0.975)
VO2 Peak (absolute)                : 1.90 L/min
VO2 Peak (relative)                : 24.99 mL/kg/min
Peak HR                            : 0.00 bpm
Peak VE                            : 49.71 L/min
Peak VE/VO2                        : 26.17 
Peak O2 Pulse                      : nan mL/beat
Peak RER                           : 0.94 
Peak Respiratory Rate              : 33.08 breaths/min
Peak Tidal Volume                  : 1.50 L
PetCO2 at VO2 Peak                 : Column missing
PetCO2 at AT (V-slope)             : Column missing
PetCO2 at AT (VE/VO2)              : Column missing


# FMD

In [15]:
# Import the FMD dataset as df_fmd

filename = "PCS04_V2_FMD.csv"

df_fmd2 = pd.read_csv(filename, engine='python')


df_fmd = pd.read_csv(filename, skiprows=7, skipfooter=2, engine='python')

df_fmd.head()

,Time [min:sec],Mean Diameter [mm],Positive Shear Rate [sec-1],Negative Shear Rate [sec-1],Positive Velocity [cm/sec],Negative Velocity [cm/sec],Unnamed: 6,Time [ms],Instant Diameter [mm],Mean Diameter [mm].1,Positive Velocity [cm/sec].1,Negative Velocity [cm/sec].1
0,00:00,4.270,308.9,-61.9,32.97,-6.61,NaN,0,4.273,4.270,32.97,-6.61
1,00:01,4.270,284.2,-68.8,30.34,-7.34,NaN,33,4.271,4.270,32.88,-6.64
2,00:02,4.269,268.7,-68.9,28.67,-7.35,NaN,66,4.278,4.270,32.82,-6.65
3,00:03,4.269,267.8,-70.4,28.58,-7.51,NaN,100,4.279,4.271,32.78,-6.67
4,00:04,4.268,262.1,-70.2,27.96,-7.49,NaN,133,4.275,4.271,32.70,-6.70


In [16]:
# Convert 'Time [min:sec]' to total seconds
df_fmd[['Minutes', 'Seconds']] = df_fmd['Time [min:sec]'].str.split(':', expand=True).astype(float)
df_fmd['Time_seconds'] = df_fmd['Minutes'] * 60 + df_fmd['Seconds']
df_fmd.drop(columns=['Minutes', 'Seconds'], inplace=True)

# Remove rows with missing values in key columns
df_fmd_clean = df_fmd.dropna(subset=['Mean Diameter [mm]', 'Positive Shear Rate [sec-1]', 
                                      'Negative Shear Rate [sec-1]', 'Positive Velocity [cm/sec]', 
                                      'Negative Velocity [cm/sec]'])

# Define baseline and hyperemia periods based on time
baseline_period = df_fmd_clean[df_fmd_clean['Time_seconds'] < 60]  # First 60 sec
hyperemia_period = df_fmd_clean[(df_fmd_clean['Time_seconds'] >= 420) & (df_fmd_clean['Time_seconds'] < 480)]  # 7 to 8 min

# Diameter Baseline
diameter_baseline = baseline_period['Mean Diameter [mm]'].mean() / 10  # Convert mm to cm

# Diameter Maximum
df_fmd['Mean Diameter [mm]'] = pd.to_numeric(df_fmd['Mean Diameter [mm]'], errors='coerce')
diameter_max = hyperemia_period['Mean Diameter [mm]'].max() / 10  # Convert mm to cm

# FMD Percent
fmd_percent = ((diameter_max - diameter_baseline) / diameter_baseline) * 100

# Shear Rate
shear_rate_baseline = df_fmd2['Positive Shear Rate Baseline [sec-1]'][0]
shear_rate_maximum = df_fmd2['Positive Shear Rate Maximum [sec-1]'][0]
shear_rate_areatomaximum = df_fmd2['Positive Shear Rate Area to Maximum []'][0]

# Hyperemia Shear Rate Max AUC
shear_rate_auc = trapz(4 * hyperemia_period['Positive Velocity [cm/sec]'] / hyperemia_period['Mean Diameter [mm]'])

# Mean Velocity
velocity_baseline = df_fmd2['Positive Velocity Baseline [cm/sec]'][0]
velocity_maximum = df_fmd2['Positive Velocity Maximum [cm/sec]'][0]
velocity_mean = df_fmd['Positive Velocity [cm/sec]'].mean()

# Flow Rate
flow_rate = (3.1416 * ((df_fmd['Mean Diameter [mm]'] / 10) / 2) ** 2 * df_fmd['Positive Velocity [cm/sec]'] * 60).mean()

# FMD Normalized
fmd_normalized = fmd_percent / shear_rate_auc

print(f"Diameter Baseline (cm): {diameter_baseline:.3f}")
print(f"Diameter Maximum (cm): {diameter_max:.3f}")
print(f"FMD (%): {fmd_percent:.2f}")
print(f"Shear Rate Baseline (s⁻¹): {shear_rate_baseline:.2f}")
print(f"Shear Rate Maximum (s⁻¹): {shear_rate_maximum:.2f}")
print(f"Shear Rate Area to Maximum (s⁻¹): {shear_rate_areatomaximum:.2f}")
print(f"Hyperemia Shear Rate Max (AUC): {shear_rate_auc:.2f}")
print(f"Baseline Velocity (cm/s): {velocity_baseline:.2f}")
print(f"Maximum Velocity (cm/s): {velocity_maximum:.2f}")
print(f"Mean Velocity (cm/s): {velocity_mean:.2f}")
print(f"Flow Rate (ml/min): {flow_rate:.2f}")
print(f"FMD Normalized: {fmd_normalized:.5f}")


Diameter Baseline (cm): 0.427
Diameter Maximum (cm): 0.475
FMD (%): 11.32
Shear Rate Baseline (s⁻¹): 325.15
Shear Rate Maximum (s⁻¹): 894.22
Shear Rate Area to Maximum (s⁻¹): 19562.50
Hyperemia Shear Rate Max (AUC): 2197.76
Baseline Velocity (cm/s): 34.70
Maximum Velocity (cm/s): 97.44
Mean Velocity (cm/s): 29.97
Flow Rate (ml/min): 282.98
FMD Normalized: 0.00515


C:\Users\ZCooper\AppData\Local\Temp\ipykernel_37476\1023748718.py:31: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  shear_rate_auc = trapz(4 * hyperemia_period['Positive Velocity [cm/sec]'] / hyperemia_period['Mean Diameter [mm]'])


[]


# PetCo2

In [35]:
import pandas as pd
import numpy as np

# Assuming your dataframe has a column 'FECO2' (fractional CO2)
baro_pressure = 762.4  # mmHg
ph2o = 47  # mmHg
vslope =  1.092734814
vem = 0.575460732 

# Calculate PETCO2 (assuming FECO2 is in %)
df_cpet['PETCO2'] = (df_cpet['FECO2'] / 100) * (baro_pressure - ph2o)

# --- 1. PETCO2 at Peak VO2 ---
max_vo2_idx = df_cpet['VO2'].idxmax()
petco2_at_max_vo2 = df_cpet.loc[max_vo2_idx, 'PETCO2']

# --- 2. PETCO2 at AT (V-slope method) ---
# Assuming 'vslope' is the index/row where V-slope threshold occurs
petco2_at_vslope = df_cpet.loc[vslope, 'PETCO2']  # Replace 'vslope' with your actual threshold index

# --- 3. PETCO2 at AT (Ventilatory Equivalent Method - VEM) ---
# Assuming 'vem' is the index/row where VE/VO2 threshold occurs
petco2_at_vem = df_cpet.loc[vem, 'PETCO2']  # Replace 'vem' with your actual threshold index

# --- Print Results ---
print(f"PETCO2 at peak VO2: {petco2_at_max_vo2:.1f} mmHg")
print(f"PETCO2 at AT (V-slope): {petco2_at_vslope:.1f} mmHg")
print(f"PETCO2 at AT (VE/VO2 method): {petco2_at_vem:.1f} mmHg")


KeyError: 1.092734814

# End